In [1]:
import tensorflow as tf
import tensornets as nets
import cv2
import numpy as np
import time
import tensornets.datasets.coco as coco

In [2]:
inputs = tf.placeholder(tf.float32, [None, 416, 416, 3]) 
model = nets.YOLOv3COCO(inputs, nets.Darknet19)

W1102 20:15:53.560917 10840 deprecation_wrapper.py:119] From I:\Anaconda\envs\neural\lib\site-packages\tensornets\utils.py:238: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W1102 20:15:53.773361 10840 deprecation_wrapper.py:119] From I:\Anaconda\envs\neural\lib\site-packages\tensornets\utils.py:283: The name tf.get_variable_scope is deprecated. Please use tf.compat.v1.get_variable_scope instead.

W1102 20:15:56.477125 10840 deprecation_wrapper.py:119] From I:\Anaconda\envs\neural\lib\site-packages\tensornets\utils.py:246: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1102 20:15:56.478124 10840 deprecation_wrapper.py:119] From I:\Anaconda\envs\neural\lib\site-packages\tensornets\utils.py:125: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.



In [3]:
imge = np.array(nets.utils.load_img('YOLO_test.jpg', target_size=416, crop_size=416))
img_to_show = cv2.cvtColor(np.uint8(imge[0]), cv2.COLOR_RGB2BGR)
cv2.imshow("image", img_to_show)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
def yolo(sess, image, threshold, classnames):    
    image = model.preprocess(image)    
    preds = sess.run(model.preds, {inputs: image})
    boxes = np.array(model.get_boxes(preds, image.shape[1:3]))
    img_to_show = cv2.cvtColor(np.uint8(imge[0]), cv2.COLOR_RGB2BGR)
    for index, cls in enumerate(classnames):
        if (len(boxes[index]) == 0) :
            continue
        for obj_box in boxes[index]: 
            if  (obj_box[4] > threshold) :                
                cv2.rectangle(img_to_show,(obj_box[0],obj_box[1]),(obj_box[2],obj_box[3]),(0,255,0),1)
                cv2.putText(img_to_show, "{} {:.2f}".format(cls, obj_box[4]), (obj_box[0],obj_box[1]), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), lineType=cv2.LINE_AA)
            else: continue
    return img_to_show

In [5]:
imge = np.array(nets.utils.load_img('YOLO_test.jpg', target_size=416, crop_size=416))
with tf.Session() as sess:
    sess.run(model.pretrained())
    result = yolo(sess, imge, 0.8 ,coco.classnames)

cv2.imshow("image", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

W1102 20:15:59.795580 10840 deprecation_wrapper.py:119] From I:\Anaconda\envs\neural\lib\site-packages\tensornets\utils.py:130: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.



In [22]:
def crop_square(image):
    y=image.shape[0]
    x=image.shape[1]
    x=(x/2) - y/2
    int(x)
    frame = image[0:int(y), int(x):int(x+y)]
    return frame

In [24]:
with tf.Session() as sess:
    sess.run(model.pretrained())    
    cap = cv2.VideoCapture("yolo_video.webm")
    while(cap.isOpened()): 
        ret, frame = cap.read()
        if(not ret):
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = crop_square(frame)
        image_to_show = cv2.resize(frame,(416,416))
        imge = np.array(image_to_show).reshape(-1,416,416,3)
        img = yolo(sess, imge, 0.5, coco.classnames)  
        cv2.imshow("image",img)  
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break          

cap.release()
cv2.destroyAllWindows() 